In [5]:
import tensorflow as tf 
from Vnet_multiclass import FillWithBone
from Unet_modified import CreateUnetModified
import os
from sklearn.model_selection import train_test_split
import nibabel as nib
import numpy as np 
import matplotlib.pyplot as plt 

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [6]:
rootDirPath = "..\\mmwhs\\ct_train"
pathList = os.listdir(rootDirPath)

In [ ]:
a = np.array(nib.load("E:\isef-vascular\VascuIAR\DeepLearning\mmwhs\ct_train\ct_train_1001_image.nii.gz").get_fdata())

with open(".\\temp.txt", "w") as f: 
    f.write("First view\n")
    for i in range(a.shape[0]): 
        u = np.unique(a[i, :, :])
        f.write(f"Slice {i} has {len(u)} unique values which are: {u}\n")
    del i 

    f.write("\nSecond view\n")
    for i in range(a.shape[1]): 
        u = np.unique(a[:, i, :])
        f.write(f"Slice {i} has {len(u)} unique values which are: {u}\n")
    del i 

    f.write("\nThird view\n")
    for i in range(a.shape[2]): 
        u = np.unique(a[:, :, i])
        f.write(f"Slice {i} has {len(u)} unique values which are: {u}\n")
    del i 

### EACH VIEW HAS DIFFERENT UNIQUE VALUES => INCONSISTENT WHEN TRYING TO REMOVE SLICES WITHOUT HEART + BY REMOVING SLICES WITHOUT HEART, IT WEAKENS THE MODEL'S ABILITY TO DIFFRENTIATE STRUCTUERS LIKE BONES / AIR IN SLICES CONTAINING HEART. 

# => 2 SOLUTIONS: 
#   1) Fill labels with zeros and real data with bone structures to make everything the same size
#   2) Crop all slices without heart to make it the same size  

# Cropping and Adding all add biases and unbalance to the dataset, except cropping is much harder to do homologously => Fill and offset bias 

In [10]:
import nibabel as nib
import numpy as np

target = 368

def GenerateData(start, stop, pathList, target): 
  # target is 368 to allow Vnet to compress and decompress
  X = []
  Y = []
  for i in range(start, stop, 2):
      imagePath = os.path.join(rootDirPath, pathList[i])
      labelPath = os.path.join(rootDirPath, pathList[i + 1])

      image = np.array(nib.load(imagePath).get_fdata(), dtype="int8")
      label = np.array(nib.load(labelPath).get_fdata(), dtype="int8")

      image, label = FillWithBone(image, label, target)

      image = np.expand_dims(image, -1)
      label = np.expand_dims(label, -1)

      print(f"Image shape: {image.shape} || Path: {imagePath}")
      print(f"Label shape : {label.shape} || Path: {labelPath}")

      X.append(image)
      Y.append(label)

  X = np.array(X)
  Y = np.array(Y)

  del image
  del label 
  del imagePath
  del labelPath

  return X, Y
  

def TrainingLoop(model, target): 
  for i in range(0, 30, 2): 
    X, Y = GenerateData(i, i + 2, pathList, target)
    model.fit(X, Y, epochs=1, verbose=2)
    del X
    del Y


In [11]:
unet = CreateUnetModified()

In [12]:
TrainingLoop(unet, target)

Image shape: (512, 512, 368, 1) || Path: ..\mmwhs\ct_train\ct_train_1001_image.nii.gz
Label shape : (512, 512, 368, 1) || Path: ..\mmwhs\ct_train\ct_train_1001_label.nii.gz
